In [306]:
import pandas as pd
pd.set_option('display.max_columns', None)  # show all columns

games = pd.read_csv("../data/raw/all_games.csv")
games = games[(games["SEASON_ID"] >= 22010) & (games["SEASON_ID"] <= 22030)]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22025,1610612737,ATL,Atlanta Hawks,22501217,2025-12-14,ATL vs. PHI,W,240,120,43,99,0.434,17,33.0,0.515,17,21,0.810,18.0,39.0,57.0,33,8.0,6,16,23,3.0
1,22025,1610612737,ATL,Atlanta Hawks,22501210,2025-12-12,ATL @ DET,L,241,115,39,85,0.459,11,36.0,0.306,26,31,0.839,13.0,24.0,37.0,28,9.0,8,20,18,-27.0
2,22025,1610612737,ATL,Atlanta Hawks,22500351,2025-12-06,ATL @ WAS,W,241,131,52,98,0.531,13,39.0,0.333,14,14,1.000,15.0,30.0,45.0,36,13.0,4,12,16,15.0
3,22025,1610612737,ATL,Atlanta Hawks,22500340,2025-12-05,ATL vs. DEN,L,241,133,49,95,0.516,20,46.0,0.435,15,16,0.938,11.0,34.0,45.0,35,8.0,2,12,25,-1.0
4,22025,1610612737,ATL,Atlanta Hawks,22500327,2025-12-03,ATL vs. LAC,L,241,92,34,85,0.400,13,39.0,0.333,11,14,0.786,7.0,31.0,38.0,20,9.0,1,16,17,-23.0


In [307]:
games["HOME_TEAM"] = games["MATCHUP"].str.contains("vs").astype(int)
games["POSS"] = 0.96 * (games["FGA"] - games["OREB"] + games["TOV"] + 0.44 * games["FTA"])
games["OFF_RTG"] = 100 * (games["PTS"] / games["POSS"])
games["TS_PCT"] = games["PTS"] / (2 * (games["FGA"] + 0.44 * games["FTA"]))
games["EFG_PCT"] = (games["FGM"] + 0.5 * games["FG3M"]) / games["FGA"]
games["A_TO_RATIO"] = games["AST"] / games["TOV"]
games["TO_PCT"] = games["TOV"] / (games["FGA"] + 0.44 * games["FTA"] + games["TOV"])
games["FT_RATE"] = games["FTA"] / games["FGA"]
games["WIN_FLAG"]  = (games["WL"] == "W").astype(int)

games["GAME_DATE"] = pd.to_datetime(games["GAME_DATE"])
games = games.sort_values(["TEAM_ID", "GAME_DATE"])

games["REST_DAYS"] = games.groupby("TEAM_ID")["GAME_DATE"].diff().dt.days
games["REST_DAYS"] = games["REST_DAYS"].fillna(7)
games["REST_DAYS"] = games["REST_DAYS"].clip(lower=0,upper=7)

games["B2B"] = (games["REST_DAYS"] <= 1).astype(int)

games["WIN_FLAG_SHIFTED"] = games.groupby("TEAM_ID")["WIN_FLAG"].shift(1)
games["WIN_STREAK"] = 0
for team_id, group in games.groupby("TEAM_ID"):
    streak = 0
    indices = group.index
    
    for idx in indices:
        past_win = games.loc[idx, "WIN_FLAG_SHIFTED"]
        if past_win == 1:
            streak += 1
        else:
            streak = 0
        games.loc[idx, "WIN_STREAK"] = streak

games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HOME_TEAM,POSS,OFF_RTG,TS_PCT,EFG_PCT,A_TO_RATIO,TO_PCT,FT_RATE,WIN_FLAG,REST_DAYS,B2B,WIN_FLAG_SHIFTED,WIN_STREAK
1390,22010,1610612737,ATL,Atlanta Hawks,21000008,2010-10-27,ATL @ MEM,W,240,119,42,80,0.525,8,16.0,0.500,27,31,0.871,9.0,35.0,44.0,20,7.0,9,15,20,15.0,0,95.6544,124.406196,0.635412,0.575000,1.333333,0.138071,0.387500,1,7.0,0,NaN,0
1389,22010,1610612737,ATL,Atlanta Hawks,21000021,2010-10-29,ATL @ PHI,W,241,104,36,70,0.514,6,17.0,0.353,26,31,0.839,10.0,36.0,46.0,24,3.0,8,20,26,3.0,0,89.8944,115.691300,0.621712,0.557143,1.200000,0.192976,0.442857,1,2.0,0,1.0,1
1388,22010,1610612737,ATL,Atlanta Hawks,21000031,2010-10-30,ATL vs. WAS,W,239,99,34,81,0.420,5,19.0,0.263,26,30,0.867,13.0,30.0,43.0,19,4.0,4,16,22,4.0,1,93.3120,106.095679,0.525478,0.450617,1.187500,0.145191,0.370370,1,1.0,1,1.0,2
1387,22010,1610612737,ATL,Atlanta Hawks,21000047,2010-11-02,ATL @ CLE,W,242,100,37,84,0.440,5,17.0,0.294,21,30,0.700,14.0,29.0,43.0,26,7.0,7,7,17,12.0,0,86.5920,115.484109,0.514403,0.470238,3.714286,0.067179,0.357143,1,3.0,0,1.0,3
1386,22010,1610612737,ATL,Atlanta Hawks,21000054,2010-11-03,ATL vs. DET,W,240,94,36,67,0.537,3,14.0,0.214,19,23,0.826,3.0,32.0,35.0,25,3.0,3,13,13,9.0,1,83.6352,112.392868,0.609440,0.559701,1.923077,0.144252,0.343284,1,1.0,1,1.0,4


In [308]:
games.sort_values(["TEAM_ID", "GAME_DATE"], inplace=True, ascending=[True, True])
# choose which columns to compute rolling averages for
rolling_features = [
    "OFF_RTG",
    "EFG_PCT",
    "TS_PCT",
    "TO_PCT",
    "FT_RATE",
    "REB",
    "A_TO_RATIO",
    "WIN_FLAG",
    "BLK",
    "STL",
    "OREB",
    "PF",
    "POSS",
    "PLUS_MINUS"
]

# how many previous games to use
window = 10

# create rolling‑window features
for col in rolling_features:
    new_col = f"{col}_L{window}"
    games[new_col] = (
        games.groupby("TEAM_ID")[col]
        .transform(lambda s: s.shift().rolling(window, min_periods=1).mean())
    )

print([f"{c}_L{window}" for c in rolling_features])
games.head()

['OFF_RTG_L10', 'EFG_PCT_L10', 'TS_PCT_L10', 'TO_PCT_L10', 'FT_RATE_L10', 'REB_L10', 'A_TO_RATIO_L10', 'WIN_FLAG_L10', 'BLK_L10', 'STL_L10', 'OREB_L10', 'PF_L10', 'POSS_L10', 'PLUS_MINUS_L10']


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HOME_TEAM,POSS,OFF_RTG,TS_PCT,EFG_PCT,A_TO_RATIO,TO_PCT,FT_RATE,WIN_FLAG,REST_DAYS,B2B,WIN_FLAG_SHIFTED,WIN_STREAK,OFF_RTG_L10,EFG_PCT_L10,TS_PCT_L10,TO_PCT_L10,FT_RATE_L10,REB_L10,A_TO_RATIO_L10,WIN_FLAG_L10,BLK_L10,STL_L10,OREB_L10,PF_L10,POSS_L10,PLUS_MINUS_L10
1390,22010,1610612737,ATL,Atlanta Hawks,21000008,2010-10-27,ATL @ MEM,W,240,119,42,80,0.525,8,16.0,0.500,27,31,0.871,9.0,35.0,44.0,20,7.0,9,15,20,15.0,0,95.6544,124.406196,0.635412,0.575000,1.333333,0.138071,0.387500,1,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1389,22010,1610612737,ATL,Atlanta Hawks,21000021,2010-10-29,ATL @ PHI,W,241,104,36,70,0.514,6,17.0,0.353,26,31,0.839,10.0,36.0,46.0,24,3.0,8,20,26,3.0,0,89.8944,115.691300,0.621712,0.557143,1.200000,0.192976,0.442857,1,2.0,0,1.0,1,124.406196,0.575000,0.635412,0.138071,0.387500,44.000000,1.333333,1.0,9.0,7.000000,9.000000,20.000000,95.6544,15.000000
1388,22010,1610612737,ATL,Atlanta Hawks,21000031,2010-10-30,ATL vs. WAS,W,239,99,34,81,0.420,5,19.0,0.263,26,30,0.867,13.0,30.0,43.0,19,4.0,4,16,22,4.0,1,93.3120,106.095679,0.525478,0.450617,1.187500,0.145191,0.370370,1,1.0,1,1.0,2,120.048748,0.566071,0.628562,0.165523,0.415179,45.000000,1.266667,1.0,8.5,5.000000,9.500000,23.000000,92.7744,9.000000
1387,22010,1610612737,ATL,Atlanta Hawks,21000047,2010-11-02,ATL @ CLE,W,242,100,37,84,0.440,5,17.0,0.294,21,30,0.700,14.0,29.0,43.0,26,7.0,7,7,17,12.0,0,86.5920,115.484109,0.514403,0.470238,3.714286,0.067179,0.357143,1,3.0,0,1.0,3,115.397725,0.527587,0.594201,0.158746,0.400243,44.333333,1.240278,1.0,7.0,4.666667,10.666667,22.666667,92.9536,7.333333
1386,22010,1610612737,ATL,Atlanta Hawks,21000054,2010-11-03,ATL vs. DET,W,240,94,36,67,0.537,3,14.0,0.214,19,23,0.826,3.0,32.0,35.0,25,3.0,3,13,13,9.0,1,83.6352,112.392868,0.609440,0.559701,1.923077,0.144252,0.343284,1,1.0,1,1.0,4,115.419321,0.513250,0.574251,0.135854,0.389468,44.000000,1.858780,1.0,7.0,5.250000,11.500000,21.250000,91.3632,8.500000


In [309]:
opp_df = games.copy()
opp_df["HOME_TEAM"] = 1 - opp_df["HOME_TEAM"]

merged = games.merge(opp_df, on="GAME_ID", suffixes=("", "_OPP"))
merged = merged[merged["TEAM_ID"] != merged["TEAM_ID_OPP"]]

merged["DEF_RTG"] = 100 * (merged["PTS_OPP"] / merged["POSS_OPP"])
merged["NET_RTG"] = merged["OFF_RTG"] - merged["DEF_RTG"]
merged["DEF_RTG_OPP"] = 100 * (merged["PTS"] / merged["POSS"])
merged["NET_RTG_OPP"] = merged["OFF_RTG_OPP"] - merged["DEF_RTG_OPP"]
#merged["PACE"]      = 48 * (merged["POSS"] + merged["POSS_OPP"]) / (2 * merged["MIN"])
#FIX PACE CALCULATION
merged["TS_DIFF"]   = merged["TS_PCT"] - merged["TS_PCT_OPP"]
merged.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HOME_TEAM,POSS,OFF_RTG,TS_PCT,EFG_PCT,A_TO_RATIO,TO_PCT,FT_RATE,WIN_FLAG,REST_DAYS,B2B,WIN_FLAG_SHIFTED,WIN_STREAK,OFF_RTG_L10,EFG_PCT_L10,TS_PCT_L10,TO_PCT_L10,FT_RATE_L10,REB_L10,A_TO_RATIO_L10,WIN_FLAG_L10,BLK_L10,STL_L10,OREB_L10,PF_L10,POSS_L10,PLUS_MINUS_L10,SEASON_ID_OPP,TEAM_ID_OPP,TEAM_ABBREVIATION_OPP,TEAM_NAME_OPP,GAME_DATE_OPP,MATCHUP_OPP,WL_OPP,MIN_OPP,PTS_OPP,FGM_OPP,FGA_OPP,FG_PCT_OPP,FG3M_OPP,FG3A_OPP,FG3_PCT_OPP,FTM_OPP,FTA_OPP,FT_PCT_OPP,OREB_OPP,DREB_OPP,REB_OPP,AST_OPP,STL_OPP,BLK_OPP,TOV_OPP,PF_OPP,PLUS_MINUS_OPP,HOME_TEAM_OPP,POSS_OPP,OFF_RTG_OPP,TS_PCT_OPP,EFG_PCT_OPP,A_TO_RATIO_OPP,TO_PCT_OPP,FT_RATE_OPP,WIN_FLAG_OPP,REST_DAYS_OPP,B2B_OPP,WIN_FLAG_SHIFTED_OPP,WIN_STREAK_OPP,OFF_RTG_L10_OPP,EFG_PCT_L10_OPP,TS_PCT_L10_OPP,TO_PCT_L10_OPP,FT_RATE_L10_OPP,REB_L10_OPP,A_TO_RATIO_L10_OPP,WIN_FLAG_L10_OPP,BLK_L10_OPP,STL_L10_OPP,OREB_L10_OPP,PF_L10_OPP,POSS_L10_OPP,PLUS_MINUS_L10_OPP,DEF_RTG,NET_RTG,DEF_RTG_OPP,NET_RTG_OPP,TS_DIFF
1,22010,1610612737,ATL,Atlanta Hawks,21000008,2010-10-27,ATL @ MEM,W,240,119,42,80,0.525,8,16.0,0.500,27,31,0.871,9.0,35.0,44.0,20,7.0,9,15,20,15.0,0,95.6544,124.406196,0.635412,0.575000,1.333333,0.138071,0.387500,1,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22010,1610612763,MEM,Memphis Grizzlies,2010-10-27,MEM vs. ATL,L,240,104,40,86,0.465,4,9.0,0.444,20,25,0.800,9.0,30.0,39.0,13,7.0,4,14,26,-15.0,0,97.9200,106.209150,0.536082,0.488372,0.928571,0.126126,0.290698,0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.209150,18.197045,124.406196,-18.197045,0.099330
3,22010,1610612737,ATL,Atlanta Hawks,21000021,2010-10-29,ATL @ PHI,W,241,104,36,70,0.514,6,17.0,0.353,26,31,0.839,10.0,36.0,46.0,24,3.0,8,20,26,3.0,0,89.8944,115.691300,0.621712,0.557143,1.200000,0.192976,0.442857,1,2.0,0,1.0,1,124.406196,0.575000,0.635412,0.138071,0.387500,44.000000,1.333333,1.0,9.0,7.000000,9.000000,20.000000,95.6544,15.000000,22010,1610612755,PHI,Philadelphia 76ers,2010-10-29,PHI vs. ATL,L,241,101,35,85,0.412,5,11.0,0.455,26,33,0.788,12.0,24.0,36.0,24,9.0,6,8,21,-3.0,0,91.6992,110.142728,0.507436,0.441176,3.000000,0.074405,0.388235,0,2.0,0,0.0,0,96.573956,0.475904,0.495219,0.154083,0.132530,38.0,1.500000,0.000000,4.000000,10.000000,10.0,22.000000,90.0864,-10.00,110.142728,5.548572,115.691300,-5.548572,0.114276
5,22010,1610612737,ATL,Atlanta Hawks,21000031,2010-10-30,ATL vs. WAS,W,239,99,34,81,0.420,5,19.0,0.263,26,30,0.867,13.0,30.0,43.0,19,4.0,4,16,22,4.0,1,93.3120,106.095679,0.525478,0.450617,1.187500,0.145191,0.370370,1,1.0,1,1.0,2,120.048748,0.566071,0.628562,0.165523,0.415179,45.000000,1.266667,1.0,8.5,5.000000,9.500000,23.000000,92.7744,9.000000,22010,1610612764,WAS,Washington Wizards,2010-10-30,WAS @ ATL,L,241,95,38,76,0.500,4,12.0,0.333,15,21,0.714,8.0,26.0,34.0,16,8.0,13,17,25,-4.0,1,90.4704,105.006720,0.557250,0.526316,0.941176,0.166275,0.276316,0,2.0,0,0.0,0,85.433136,0.397436,0.455044,0.124760,0.384615,25.0,1.307692,0.000000,3.000000,10.000000,3.0,25.000000,97.1520,-29.00,105.006720,1.088959,106.095679,-1.088959,-0.031772
7,22010,1610612737,ATL,Atlanta Hawks,21000047,2010-11-02,ATL @ CLE,W,242,100,37,84,0.440,5,17.0,0.294,21,30,0.700,14.0,29.0,43.0,26,7.0,7,7,17,12.0,0,86.5920,115.484109,0.514403,0.470238,3.714286,0.067179,0.357143,1,3.0,0,1.0,3,115.397725,0.527587,0.594201,0.158746,0.400243,44.333333,1.240278,1.0,7.0,4.666667,10.666667,22.666667,92.9536,7.333333,22010,1610612739,CLE,Cleveland Cavaliers,2010-11-02,CLE vs. ATL,L,239,88,32,74,0.432,5,17.0,0.294,19,26,0.731,10.0,34.0,44.0,15,2.0,4,14,22,-12.0,0,85.8624,102.489565,0.514981,0.466216,1.071429,0.140788,0.351351,0,3.0,0,0.0,0,100.567178,0.480335,0.515035,0.144711,0.246936,35.0,1.398689,0.333333,2.666667,7.333333,9.0,19.333333,92.9280,-5.00,102.489565,12.994545,115.484109,-12.994545,-0.000578
9,22010,1610612737,ATL,Atla

In [310]:
merged.sort_values(["TEAM_ID", "GAME_DATE"], inplace=True, ascending=[True, True])
merged["DEF_RTG_L10"] = merged.groupby("TEAM_ID")["DEF_RTG"].transform(
    lambda s: s.shift().rolling(window).mean()
)
merged["DEF_RTG_L10_OPP"] = merged.groupby("TEAM_ID")["DEF_RTG_OPP"].transform(
    lambda s: s.shift().rolling(window).mean()
)
merged["NET_RTG_L10"] = merged["OFF_RTG_L10"] - merged["DEF_RTG_L10"]
merged["NET_RTG_L10_OPP"] = merged["OFF_RTG_L10_OPP"] - merged["DEF_RTG_L10_OPP"]
merged.sort_values(["TEAM_ID", "GAME_DATE"], inplace=True, ascending=[True, False])
merged.dropna(inplace=True)
merged

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HOME_TEAM,POSS,OFF_RTG,TS_PCT,EFG_PCT,A_TO_RATIO,TO_PCT,FT_RATE,WIN_FLAG,REST_DAYS,B2B,WIN_FLAG_SHIFTED,WIN_STREAK,OFF_RTG_L10,EFG_PCT_L10,TS_PCT_L10,TO_PCT_L10,FT_RATE_L10,REB_L10,A_TO_RATIO_L10,WIN_FLAG_L10,BLK_L10,STL_L10,OREB_L10,PF_L10,POSS_L10,PLUS_MINUS_L10,SEASON_ID_OPP,TEAM_ID_OPP,TEAM_ABBREVIATION_OPP,TEAM_NAME_OPP,GAME_DATE_OPP,MATCHUP_OPP,WL_OPP,MIN_OPP,PTS_OPP,FGM_OPP,FGA_OPP,FG_PCT_OPP,FG3M_OPP,FG3A_OPP,FG3_PCT_OPP,FTM_OPP,FTA_OPP,FT_PCT_OPP,OREB_OPP,DREB_OPP,REB_OPP,AST_OPP,STL_OPP,BLK_OPP,TOV_OPP,PF_OPP,PLUS_MINUS_OPP,HOME_TEAM_OPP,POSS_OPP,OFF_RTG_OPP,TS_PCT_OPP,EFG_PCT_OPP,A_TO_RATIO_OPP,TO_PCT_OPP,FT_RATE_OPP,WIN_FLAG_OPP,REST_DAYS_OPP,B2B_OPP,WIN_FLAG_SHIFTED_OPP,WIN_STREAK_OPP,OFF_RTG_L10_OPP,EFG_PCT_L10_OPP,TS_PCT_L10_OPP,TO_PCT_L10_OPP,FT_RATE_L10_OPP,REB_L10_OPP,A_TO_RATIO_L10_OPP,WIN_FLAG_L10_OPP,BLK_L10_OPP,STL_L10_OPP,OREB_L10_OPP,PF_L10_OPP,POSS_L10_OPP,PLUS_MINUS_L10_OPP,DEF_RTG,NET_RTG,DEF_RTG_OPP,NET_RTG_OPP,TS_DIFF,DEF_RTG_L10,DEF_RTG_L10_OPP,NET_RTG_L10,NET_RTG_L10_OPP
2431,22025,1610612737,ATL,Atlanta Hawks,22501217,2025-12-14,ATL vs. PHI,W,240,120,43,99,0.434,17,33.0,0.515,17,21,0.810,18.0,39.0,57.0,33,8.0,6,16,23,3.0,1,101.9904,117.658133,0.554324,0.520202,2.062500,0.128783,0.212121,1,2.0,0,0.0,0,118.065223,0.552028,0.582113,0.118280,0.219959,41.8,2.576264,0.5,4.8,9.1,10.7,19.3,100.20096,-2.1,22025,1610612755,PHI,Philadelphia 76ers,2025-12-14,PHI @ ATL,L,240,117,38,101,0.376,17,41.0,0.415,24,29,0.828,21.0,33.0,54.0,25,10.0,10,11,15,-3.0,1,99.6096,117.458558,0.514241,0.460396,2.272727,0.088169,0.287129,0,2.0,0,1.0,1,114.110054,0.512907,0.558838,0.117492,0.270578,46.8,2.040757,0.6,5.4,9.2,11.7,19.9,101.05728,0.3,117.458558,0.199574,117.658133,-0.199574,0.040083,119.908722,118.065223,-1.843499,-3.955170
2429,22025,1610612737,ATL,Atlanta Hawks,22501210,2025-12-12,ATL @ DET,L,241,115,39,85,0.459,11,36.0,0.306,26,31,0.839,13.0,24.0,37.0,28,9.0,8,20,18,-27.0,0,101.4144,113.396125,0.582928,0.523529,1.400000,0.168577,0.364706,0,6.0,0,1.0,1,119.503064,0.567491,0.591766,0.116139,0.198431,42.0,2.623764,0.5,4.6,8.9,10.0,19.9,99.92064,-0.3,22025,1610612765,DET,Detroit Pistons,2025-12-12,DET vs. ATL,W,240,142,53,91,0.582,12,29.0,0.414,24,29,0.828,10.0,30.0,40.0,34,16.0,6,12,22,27.0,0,101.5296,139.860691,0.684271,0.648352,2.833333,0.103663,0.318681,1,6.0,0,1.0,2,119.822394,0.555644,0.597163,0.139895,0.351831,45.8,1.650042,0.7,5.7,8.5,12.8,21.7,99.06048,3.8,139.860691,-26.464566,113.396125,26.464566,-0.101344,119.418334,119.503064,0.084730,0.319330
2427,22025,1610612737,ATL,Atlanta Hawks,22500351,2025-12-06,ATL @ WAS,W,241,131,52,98,0.531,13,39.0,0.333,14,14,1.000,15.0,30.0,45.0,36,13.0,4,12,16,15.0,0,97.1136,134.893568,0.628840,0.596939,3.000000,0.103306,0.142857,1,1.0,1,0.0,0,117.727228,0.564694,0.588711,0.121179,0.201386,41.0,2.506117,0.4,4.7,8.5,9.6,20.7,99.77088,-2.6,22025,1610612764,WAS,Washington Wizards,2025-12-06,WAS vs. ATL,L,240,116,41,80,0.513,17,38.0,0.447,17,19,0.895,4.0,22.0,26.0,29,6.0,5,20,18,-15.0,0,100.1856,115.785103,0.656406,0.618750,1.450000,0.184570,0.237500,0,2.0,0,0.0,0,113.221856,0.531049,0.563013,0.127056,0.244813,40.6,1.854216,0.2,5.6,7.5,10.8,20.8,97.68960,-16.3,115.785103,19.108465,134.893568,-19.108465,-0.027565,120.522454,117.727228,-2.795226,-4.505372
2425,22025,1610612737,ATL,Atlanta Hawks,22500340,2025-12-05,ATL vs. DEN,L,241,133,49,95,0.516,20,46.0,0.435,15,16,0.938,11.0,34.0,45.0,35,8.0,2,12,25,-1.0,1,98.9184,134.454257,0.651705,0.621053,2.916667,0.105226,0.168421,0,2.0,0,0.0,0,116.754443,0.563765,0.588422,0.125759,0.212779,39.9,2.420332,0.5,4.8,8.8,9.4,20.1,99.82080,-2.3,22025,1610612743,DEN,Denver Nuggets,2025-12-05,DEN @ ATL,W,239,134,47,89,0.528,16,37.0,0.432,24,30,0.800,9.0,33.0,42.0,28,9.0,6,12,15,1.0,1,100.9920,132.683777,0.655577,0.617978,2.333333,0.105079,0.337079,1,2.0,0,1.0,1,130.971657,0

In [311]:
print(merged.shape)
print(merged[["TEAM_ABBREVIATION","TEAM_ABBREVIATION_OPP"]].head(10))
print(merged.groupby("GAME_ID")["TEAM_ID"].nunique().value_counts())

(36248, 118)
     TEAM_ABBREVIATION TEAM_ABBREVIATION_OPP
2431               ATL                   PHI
2429               ATL                   DET
2427               ATL                   WAS
2425               ATL                   DEN
2423               ATL                   LAC
2421               ATL                   DET
2419               ATL                   PHI
2417               ATL                   CLE
2415               ATL                   WAS
2413               ATL                   CHA
TEAM_ID
2    18119
1       10
Name: count, dtype: int64


In [312]:
good_ids = (
    merged.groupby("GAME_ID")["TEAM_ID"]
    .nunique()
    .reset_index()
    .query("TEAM_ID == 2")["GAME_ID"]
)

merged = merged[merged["GAME_ID"].isin(good_ids)]

print(merged.shape)
print(merged.groupby("GAME_ID")["TEAM_ID"].nunique().value_counts())

(36238, 118)
TEAM_ID
2    18119
Name: count, dtype: int64


In [313]:
merged.to_csv("../data/processed/adv_nba_data.csv", index=False)